# Adversarial Training

## CIFAR-10

### IBM-ART

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

x_train = np.array([trainset[i][0].numpy() for i in range(len(trainset))])
y_train = np.array([trainset[i][1] for i in range(len(trainset))])
x_test = np.array([testset[i][0].numpy() for i in range(len(testset))])
y_test = np.array([testset[i][1] for i in range(len(testset))])

# Transpose the data to match the input shape expected by PyTorch (batch_size, 3, height, width)
x_train = x_train.transpose((0, 3, 1, 2))
x_test = x_test.transpose((0, 3, 1, 2))

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

# Initialize ResNet50 model
model = resnet50(pretrained=False)
model.fc = nn.Linear(2048, 10)  # Adjust the final layer for CIFAR-10 (10 classes)
model = model.cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from art.estimators.classification import PyTorchClassifier

# Define the ART classifier
classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
    clip_values=(0, 1)
)

In [ ]:
from art.attacks.evasion import FastGradientMethod
from art.defences.trainer import AdversarialTrainer
import time

# Define the attack
attack = FastGradientMethod(estimator=classifier, eps=0.3)

# Initialize the adversarial trainer
trainer = AdversarialTrainer(classifier, attacks=attack, ratio=0.5)

start_time = time.time()
# Perform adversarial training
trainer.fit(x_train, y_train, batch_size=128, nb_epochs=10)

end_time = time.time()
print('Adversarial training took', end_time - start_time, 'seconds')

### ARES

#### Single GPU

In [ ]:
import time
start_time = time.time()
!python -m torch.distributed.launch --nproc_per_node=1 ./ares/cifar/adversarial_training.py --configs= ./ares/cifar/resnet50.yaml
end_time = time.time()
print("Execution time: ", end_time - start_time)

#### Multi GPU

In [ ]:
import time
start_time = time.time()
!python -m torch.distributed.launch --nproc_per_node=4 ./ares/cifar/adversarial_training.py --configs= ./ares/cifar/resnet50.yaml
end_time = time.time()
print("Execution time: ", end_time - start_time)

In [ ]:
import time
start_time = time.time()
!python -m torch.distributed.launch --nproc_per_node=7 ./ares/cifar/adversarial_training.py --configs= ./ares/cifar/resnet50.yaml
end_time = time.time()
print("Execution time: ", end_time - start_time)

### AdvSecureNet

#### Single GPU

In [ ]:
import time
start_time = time.time()
!advsecurenet defense adversarial-training -c ./advsecurenet/cifar/adversarial_training_config.yml
end_time = time.time()
print("Execution time: ", end_time - start_time)

#### Multi GPU

In [ ]:
import time
start_time = time.time()
# update the config file with the number of GPUs
!advsecurenet defense adversarial-training -c ./advsecurenet/cifar/adversarial_training_config.yml
end_time = time.time()
print("Execution time: ", end_time - start_time)

## ImageNet

### IBM-ART

In [ ]:

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the transformations for the dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Path to ImageNet data
data_path = '/local/scratch/datasets/ImageNet/ILSVRC2012/'

# Load the dataset#
trainset = torchvision.datasets.ImageNet(root=data_path, split='train', transform=transform)

x_train = np.array([trainset[i][0].numpy() for i in range(len(trainset))])
y_train = np.array([trainset[i][1] for i in range(len(trainset))])


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

# Initialize ResNet50 model
model = resnet50(pretrained=False)
model = model.cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

from art.attacks.evasion import FastGradientMethod
from art.defences.trainer import AdversarialTrainer
import time

# Define the attack
attack = FastGradientMethod(estimator=classifier, eps=0.1)

# Initialize the adversarial trainer
trainer = AdversarialTrainer(classifier, attacks=attack, ratio=0.5)



start_time = time.time()
# Perform adversarial training
trainer.fit(x = x_train,
            y = y_train, 
            batch_size=32,
            nb_epochs=1)

end_time = time.time()
print('Adversarial training took', end_time - start_time, 'seconds')

### ARES

#### Single GPU

In [ ]:
import time
start_time = time.time()
!python -m torch.distributed.launch --nproc_per_node=1 ./ares/imagenet/adversarial_training.py --configs= ./ares/imagenet/resnet50.yaml
end_time = time.time()
print("Execution time: ", end_time - start_time)

#### Multi GPU

In [ ]:
import time
start_time = time.time()
!python -m torch.distributed.launch --nproc_per_node=4 ./ares/imagenet/adversarial_training.py --configs= ./ares/imagenet/resnet50.yaml
end_time = time.time()
print("Execution time: ", end_time - start_time)

In [ ]:
import time
start_time = time.time()
!python -m torch.distributed.launch --nproc_per_node=7 ./ares/imagenet/adversarial_training.py --configs= ./ares/imagenet/resnet50.yaml
end_time = time.time()
print("Execution time: ", end_time - start_time)

### AdvSecureNet

#### Single GPU

In [ ]:
import time
start_time = time.time()
!advsecurenet defense adversarial-training -c ./advsecurenet/imagenet/adversarial_training_config.yml
end_time = time.time()
print("Execution time: ", end_time - start_time)

#### Multi GPU

In [ ]:
import time
start_time = time.time()
!advsecurenet defense adversarial-training -c ./advsecurenet/imagenet/adversarial_training_config.yml
end_time = time.time()
print("Execution time: ", end_time - start_time)